In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from autobook.libgen import Libgen

data = Libgen(q="cloud cuckoo land")
df = data.search()
# df = df.filter("Anthony Doerr")

In [ ]:
df.results_df

In [ ]:
tet = df.results
tet.iloc[0:1]["md5"].values[0]


In [ ]:
df.get_filtered_results()

In [ ]:
df.download_file()

In [ ]:
from grab_fork_from_libgen import Metadata

md5 = df.get_results()["md5"][0]
meta = Metadata(timeout=(10, 20)).get_download_links(md5, topic="fiction")


In [111]:
from grab_fork_from_libgen import LibgenSearch
from grab_fork_from_libgen.exceptions import LibgenError


def filter(res, author):
    # Generate variations of the author's name
    first_name, last_name = author.split()
    author_variations = [
        f"{first_name} {last_name}",
        f"{last_name}, {first_name}",
    ]

    # Try to get the book for each variation of the author's name
    for author_name in author_variations:
        filters = {"author(s)": author_name}
        try:
            book = res.get(**filters)
            if book:
                return book
        except LibgenError:
            continue

    # If no book was found for any variation of the author's name, raise an error
    raise LibgenError("No book matches the given author.")


def filter_by_title(res, title):
    # Try to get the book for each variation of the title
    for book in res.get_results().values():
        actual_title = book["title"]
        if title in actual_title:
            return book

In [132]:
def filter_by_author_and_title(res, author=None, title=None):
    # Initialize an empty list to store matching books
    matching_books = []

    # Generate variations of the author's name if author is provided
    author_variations = []
    if author:
        first_name, last_name = author.split()
        author_variations = [
            f"{first_name} {last_name}",
            f"{last_name}, {first_name}",
        ]

    # Try to get the book for each variation of the author's name and title
    for book in res.get_results().values():
        actual_title = book["title"]
        actual_author = book["author(s)"]
        if (
            not author
            or any(
                author_name == actual_author
                for author_name in author_variations
            )
        ) and (not title or title in actual_title):
            filters = {"author(s)": actual_author, "title": actual_title}
            try:
                matching_book = res.get(save_to=, **filters)
                if matching_book:
                    matching_books.append(matching_book)
            except LibgenError:
                continue

    # If no book was found for any variation of the author's name and title, raise an error
    if not matching_books:
        raise LibgenError("No book matches the given author and/or title.")

    return matching_books


# Usage:
res = LibgenSearch("fiction", q="cloud cuckoo land")
matching_books = filter_by_author_and_title(res, author="Anthony Doerr")

In [135]:
from autobook.core.frame import LibgenDataFrame


LibgenDataFrame(matching_books).clean_title()

,author(s),series,title,language,file,mirror1,mirror2,md5,topic,extension,size
0,Anthony Doerr,,Cloud Cuckoo Land,English,EPUB / 4.98 Mb,http://library.lol/fiction/7F2FCC965C0414C4C3F...,https://library.bz/fiction/edit/7F2FCC965C0414...,7F2FCC965C0414C4C3F105FC56697F64,fiction,epub,4.98 Mb
